# Pipeline
Data Preprocessing
- Grab surface smoke data from 1-10-25 00:00 UTC to 1-17-25 00:00 UTC
- Subregion data to the LA metropolitan area
- Preprocess data into 5D tensor for convlstm input
  
Training
- Pass into convlstm
- Examine results
  
Inference
- Pass a sample into the convlstm
- Examine results

# Data Preprocessing

In [1]:
from herbie import Herbie, wgrib2, paint
from herbie.toolbox import EasyMap, ccrs, pc
from pathlib import Path
import pandas as pd
import xarray as xr
import numpy as np
import math
import cv2

In [2]:
# downloads 7 days of data (1-10 to 1-17), 24 hours (00 to 23) -> 168 hours
# not forecasts -- downloading actual hourly updated data
grib_files = []
n_frames = 0
for day in range(10, 17):
    for hour in range(0, 24):
        date = f"2025-01-{str(day).zfill(2)}-{str(hour).zfill(2)}"
        H = Herbie(
            date,
            model="hrrr",
            product="sfc",
        )
        H.download("MASSDEN")
        my_file = H.get_localFilePath("MASSDEN")
        grib_files.append(my_file)
        n_frames = n_frames + 1

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 01:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 02:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 03:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 04:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 05:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 06:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 07:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 08:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 09:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2025-Jan-10 10:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr 

In [3]:
# validate downloads, subregion each grib file, then convert to numpy arrays
lat_bottom, lat_top = 33.6, 34.3
lon_bottom, lon_top = -118.6, -117.9
extent = (lon_bottom, lon_top, lat_bottom, lat_top)

hrrr_subregion_np = np.empty((n_frames, 30, 26))
hrrr_subregion_xarr = []
for count, file in enumerate(grib_files):
    # subregion the grib files
    idx_file = wgrib2.create_inventory_file(file)    
    subset_file = wgrib2.region(file, extent, name="la_region")

    # print progress
    if ((count + 1) % 10 == 0 or count + 1 == len(grib_files)):
        print(f"{count+1}/{len(grib_files)} files converted.")

    # store as xarray and numpy
    hrrr_xarr = xr.open_dataset(subset_file, engine="cfgrib", decode_timedelta=False)
    hrrr_subregion_xarr.append(hrrr_xarr)
    hrrr_subregion_np[count] = np.flip(hrrr_xarr.mdens.to_numpy(), axis=0)

print(hrrr_subregion_np.shape)

Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_d9ef9dd4__hrrr.t00z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_87ef9dd4__hrrr.t01z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_03ef9dd4__hrrr.t02z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_64ef9dd4__hrrr.t03z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_10ef9dd4__hrrr.t04z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_47ef9dd4__hrrr.t05z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_6def9dd4__hrrr.t06z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/

10/168 files converted.


Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_5def9dd4__hrrr.t13z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_5aef9dd4__hrrr.t14z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_60ef9dd4__hrrr.t15z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_78ef9dd4__hrrr.t16z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_2aef9dd4__hrrr.t17z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_aeef9dd4__hrrr.t18z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_2bef9dd4__hrrr.t19z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/

20/168 files converted.


Ignoring index file '/home/mgraca/data/hrrr/20250110/la_region_subset_34ef9dd4__hrrr.t23z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250111/la_region_subset_d7ef9dd4__hrrr.t00z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250111/la_region_subset_c0ef9dd4__hrrr.t01z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250111/la_region_subset_86ef9dd4__hrrr.t02z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250111/la_region_subset_afef9dd4__hrrr.t03z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250111/la_region_subset_0bef9dd4__hrrr.t04z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250111/la_region_subset_13ef9dd4__hrrr.t05z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/

...

150/168 files converted.


Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_c7ef9dd4__hrrr.t09z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_35ef9dd4__hrrr.t10z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_18ef9dd4__hrrr.t11z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_b5ef9dd4__hrrr.t12z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_daef9dd4__hrrr.t13z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_8eef9dd4__hrrr.t14z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_91ef9dd4__hrrr.t15z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/

160/168 files converted.


Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_aaef9dd4__hrrr.t20z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_3bef9dd4__hrrr.t21z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_90ef9dd4__hrrr.t22z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file
Ignoring index file '/home/mgraca/data/hrrr/20250116/la_region_subset_70ef9dd4__hrrr.t23z.wrfsfcf00.grib2.5b7b6.idx' older than GRIB file


168/168 files converted.
(168, 30, 26)


In [4]:
# standard scale data
mean = hrrr_subregion_np.mean()
var = hrrr_subregion_np.var()
scaled_hrrr = (hrrr_subregion_np - mean) / math.sqrt(var)

print(f"Shape of scaled frames: {scaled_hrrr.shape}")

# decimate and create channel axis
frames = np.empty(shape=(n_frames, 40, 40, 1))
for i, frame in enumerate(scaled_hrrr):
    new_frame = cv2.resize(frame, (40, 40))
    new_frame = np.reshape(new_frame, (40, 40, 1)) # (row, col, channel)
    frames[i] = new_frame

print(f"Shape of a frame: {frames.shape}")

# create samples from frames
sample_size = 5 # aka frames per sample
n_samples = n_frames - sample_size
X = np.empty((n_samples, sample_size, 40, 40, 1))
for i in range(n_samples):
    sample = np.array([frames[j] for j in range(i, i + sample_size)])
    X[i] = sample

print(f"Shape of the final input: {X.shape}")

Shape of scaled frames: (168, 30, 26)
Shape of a frame: (168, 40, 40, 1)
Shape of the final input: (163, 5, 40, 40, 1)


# Training
```python
seq = Sequential()

seq.add(ConvLSTM2D(filters=15, kernel_size=(3, 3),
                   # rgb + graph embedding (other 2 from what?)
                   #input_shape=(5, 40, 40, 6),
                   input_shape=(12, 40, 40, 1),
                   padding='same', return_sequences=True))

seq.add(ConvLSTM2D(filters=30, kernel_size=(3, 3),
                   padding='same', return_sequences=True))

seq.add(Conv3D(filters=15, kernel_size=(3, 3, 3),
               activation='relu',
               padding='same', data_format='channels_last'))


seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='relu',
               padding='same', data_format='channels_last'))
# only necessary for predicting the 7 AirNow sensors
#seq.add(Flatten())
#seq.add(Dense(7,activation='relu'))
seq.compile(loss='mean_absolute_error', optimizer='adam')
```

# Inference